In [ ]:
# import tensorflow as tf
# tf.__version__
# os.environ["CUDA_VISIBLE_DEVICES"]="0"
import glob

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import cv2 
from keras.preprocessing import image
from keras.models import Model
from keras.optimizers import Adam
from keras.callbacks import EarlyStopping
from keras.layers import Input,Dense,Activation,BatchNormalization,Flatten,Conv2D,MaxPooling2D,UpSampling2D
from keras.layers import MaxPooling2D, Dropout, UpSampling2D
from sklearn.model_selection import train_test_split
from keras.models import load_model
from keras.utils import Sequence
from keras.callbacks import CSVLogger, EarlyStopping, ModelCheckpoint, TensorBoard
import pandas as pd
import os
from annoy import AnnoyIndex
from tqdm import tqdm_notebook as tqdm
import PIL

In [ ]:
%matplotlib inline

In [ ]:
# Check for dirs
import os
train_dir = './data/train_transformed/'
test_dir = './data/test_transformed/'
model_dir = './models/'
log_dir = './logs/'
test_batch_dir = './data/test_results'
for dir in [train_dir,test_dir,model_dir,log_dir, test_batch_dir]:
    if not(os.path.exists(dir)):
        os.mkdir(dir)

In [ ]:
# Loading info from parser

info = {
    "inputShape": (200,200,1),
    "autoencoderFile": os.path.join(model_dir, "autoencoder.h5"), 
    "encoderFile": os.path.join(model_dir, "encoder.h5"),
    "decoderFile": os.path.join(model_dir, "decoder.h5"),
    "checkpointFile": os.path.join(model_dir, "checkpoint.h5"),
    "trainHistory": os.path.join(model_dir, "train_history.csv"),
    "mode": 'train',
    "retrain": True,
    "loss": 'mse',
    "optimizer": 'adam',
    "batchSize": 32,
    "multiprocessing": True
}


In [ ]:
# Test set generation
test_datagen = image.ImageDataGenerator(rescale=1./255,
                                     rotation_range=90,
                                     width_shift_range=0.1,
                                     height_shift_range=0.1,
                                     zoom_range=0.2)

test_generator = test_datagen.flow_from_directory(
                test_dir,
                target_size=info["inputShape"][:2],
                batch_size=1,
                color_mode='grayscale',
                class_mode='input',
                seed=0
                        )

filenames = test_generator.filenames
nb_samples = len(filenames)

In [ ]:
# Train set generation (80-20 split)
train_datagen = image.ImageDataGenerator(rescale=1./255,
                                    validation_split=0.2,
                                    rotation_range=20,
                                    width_shift_range=0.2,
                                    height_shift_range=0.2,
                                    horizontal_flip=False,
                                    vertical_flip=False
                                  )
# For training (train-validation)
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=info["inputShape"][:2],
    batch_size=info["batchSize"],
    color_mode='grayscale',
    class_mode='input',
    subset='training',
    seed=0
)

validation_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=info["inputShape"][:2],
    batch_size=info["batchSize"],
    color_mode='grayscale',
    class_mode='input',
    subset='validation',
    seed=0
)

In [ ]:
class Autoencoder():
    
    def __init__(self, info):
        self.info= info

    
    def build_models(self):
        
        # Build and compile
        # Print building models
        conv_shape = (3,3) # convolutional kernel shape
        pool_shape = (2,2) # pooling kernel shape
        n_hidden_1, n_hidden_2, n_hidden_3 = 16, 8, 8 # channel numbers
        input_shape = self.info['inputShape']
        input_layer = Input(shape= input_shape)
        
        #encoder layers
        x = Conv2D(n_hidden_1, conv_shape, activation='relu', padding='same')(input_layer)
        x = MaxPooling2D(pool_shape, padding='same')(x)
        x = Conv2D(n_hidden_2, conv_shape, activation='relu', padding='same')(x)
        x = MaxPooling2D(pool_shape, padding='same')(x)
        x = Conv2D(n_hidden_3, conv_shape, activation='relu', padding='same')(x)
        encoded = MaxPooling2D(pool_shape, padding='same')(x)
        
        #decoded layers
        x = Conv2D(n_hidden_3, conv_shape, activation='relu', padding='same')(encoded)
        x = UpSampling2D(pool_shape)(x)
        x = Conv2D(n_hidden_2, conv_shape, activation='relu', padding='same')(x)
        x = UpSampling2D(pool_shape)(x)
        x = Conv2D(n_hidden_1, conv_shape, activation='relu', padding='same')(x)
        x = UpSampling2D(pool_shape)(x)
        decoded = Conv2D(input_shape[2], conv_shape, activation='sigmoid', padding='same')(x)
        
        # Creating Autoencoder
        autoencoder = Model(input_layer,decoded)
        # Creating Encoder
        encoder = Model(input_layer,encoded)
        
        # Output encoder shapes
        output_encoder_shape = encoder.layers[-1].output_shape[1:]

        # Create decoder model (Reverse)
        decoded_input = Input(shape=output_encoder_shape)
        
        decoded_output = autoencoder.layers[-7](decoded_input)  # Conv2D
        decoded_output = autoencoder.layers[-6](decoded_output)  # UpSampling2D
        decoded_output = autoencoder.layers[-5](decoded_output)  # Conv2D
        decoded_output = autoencoder.layers[-4](decoded_output)  # UpSampling2D
        decoded_output = autoencoder.layers[-3](decoded_output)  # Conv2D
        decoded_output = autoencoder.layers[-2](decoded_output)  # UpSampling2D
        decoded_output = autoencoder.layers[-1](decoded_output)  # Conv2D
        
        decoder = Model(decoded_input, decoded_output)
        
        # Generate summaries
        print("\nautoencoder.summary():")
        print(autoencoder.summary())
        print("\nencoder.summary():")
        print(encoder.summary())
        print("\ndecoder.summary():")
        print(decoder.summary())
        
        self.autoencoder = autoencoder
        self.encoder = encoder
        self.decoder = decoder
        
        print('Models succesfully built')
        
        self.compile(loss = self.info['loss'], optimizer = self.info['optimizer'])
        
        print('Building and compilation done')
        
    def compile(self, loss='binary_crossentropy', optimizer='adam'):
        print('Compiling models...')
        # To fit the model using the parameters
        self.autoencoder.compile(loss=loss,optimizer=optimizer)
    
    def predict_embedding(self,test_generator, steps):
        return self.encoder.predict_generator(test_generator, steps)
    
    def reconstruct_img(self,X):
        return self.autoencoder.predict(X)
    
    def predict_generator(self, test_generator, steps):
        return self.autoencoder.predict_generator(test_generator, steps)
    
    def fit(self, train_generator, validation_generator, n_epochs=2, batch_size=256, callbacks=[]): 
        # Split the train test set
        print('Fitting models....')

        self.autoencoder.fit_generator(
                    train_generator,
                    steps_per_epoch = train_generator.samples // batch_size,
                    validation_data = validation_generator, 
                    validation_steps = validation_generator.samples // batch_size,
                    verbose=1,
                    epochs = n_epochs,
                    use_multiprocessing=self.info["multiprocessing"],
                    callbacks=callbacks,
                    max_queue_size=16,
                    workers=8,
        )
    
    def save_models(self):
        print('Saving models...')
        self.autoencoder.save(self.info["autoencoderFile"])
        self.encoder.save(self.info["encoderFile"])
        self.decoder.save(self.info["decoderFile"])
        
        print('models.saved')
        
    def load_models(self, loss='mse', optimizer='adam'):
        print('Loading and compiling models..')
        self.autoencoder = load_model(self.info["autoencoderFile"]) #compile=False for this laptop
        self.encoder = load_model(self.info["encoderFile"])
        self.decoder = load_model(self.info["decoderFile"])
        self.autoencoder.compile(optimizer=optimizer, loss=loss)
        self.encoder.compile(optimizer=optimizer, loss=loss)
        self.decoder.compile(optimizer=optimizer, loss=loss)
        
        print('Loading and compiling models done')
        

In [ ]:
model = Autoencoder(info)

info['retrain']=False # Rebuild model for retraining

if info['retrain']:
    model.build_models()
else:

    model.load_models()
#     else:
#         model.build_models()

In [ ]:
from time import time

train_history_file = info['trainHistory']
checkpoint_file = info['checkpointFile']
csv_logger=CSVLogger(train_history_file, append=True, separator=';')
es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=20) 
checkpoint = ModelCheckpoint(checkpoint_file, monitor='val_loss', verbose=1, save_best_only=True, mode='auto', period=5)
# tensorboard = TensorBoard(log_dir=os.path.join(log_dir,str(time()))) # Tensorboard doesn't work with keras release
callbacks = [csv_logger,es,checkpoint]

In [ ]:
# Train

if info['mode']=='train':
    hist = model.fit(train_generator,
                     validation_generator, 
                     n_epochs=200,
                     batch_size=32,
                     callbacks = callbacks
             )
    model.save_models() # Save encoder models



In [ ]:
df = pd.read_csv(train_history_file,delimiter=';')
df.head()

In [ ]:
df.plot(x='epoch',y=['val_loss','loss'])

# DRAWING VISUALIZATION
## Load images from dataset

In [ ]:
# Transformation for original images
orig_datagen = image.ImageDataGenerator(rescale=1./255,)

# Generator for original images (# CHANGED TO TRAIN_DIR TEMPORARILY)
orig_generator = orig_datagen.flow_from_directory(
                train_dir,
                target_size=info["inputShape"][:2],
                batch_size=1,
                color_mode='grayscale',
                class_mode='categorical',
                seed=0,
                shuffle=False
                        )

label_names = orig_generator.filenames

In [ ]:
# Sequence for loading drawings
import math
def load_image(p):
    img = cv2.imread(p,cv2.IMREAD_GRAYSCALE)
    img = img/255.
    return img

class MySequence(Sequence):
    def __init__(self, images_dir, batch_size=1):
#       images = glob.glob(images_dir + "/**/*.jpg", recursive=True)
        images = glob.glob(images_dir + "/**/*.png", recursive=True)
        print(f"Found {len(images)} images in {images_dir}")

        self.images = images
        self.batch_size = batch_size
        self.filenames = images

    
    def on_epoch_end(self):
        #'Updates indexes after each epoch'
        self.indexes = np.arange(len(self.list_IDs))
        if self.shuffle == True:
            np.random.shuffle(self.indexes)
        
    def __len__(self):
        return math.ceil(len(self.images)/self.batch_size)

    def __getitem__(self, idx):
        # Get start and end indices for batch
        start_idx = idx * self.batch_size
        end_idx = (idx + 1) * self.batch_size

        batch_images = self.images[start_idx:end_idx]
        X = []
        img_names = ''
        for path in batch_images:
            # Load image using cv2
            img = load_image(path)

            # Resize image using cv2
            img = cv2.resize(img, (200, 200))
            
            img = img.reshape(200,200,1)
            X.append(img)
            img_name = path

        X = np.stack(X, axis=0)

        # Return the entire batch of images (you may also return the labels like so: return X, y)
        return X, img_name
    
drawings_dir = './data/drawings/'
drawing_generator = MySequence(drawings_dir)
draw_labels = drawing_generator.filenames

In [ ]:
# Calculate embeddings in the dataset
original = model.predict_embedding(orig_generator,len(label_names))  
original = original.reshape(original.shape[0],-1)

# Calculate embeddings for drawings
drawing_prob = model.predict_embedding(drawing_generator,len(draw_labels)) 
drawing_prob = drawing_prob.reshape(drawing_prob.shape[0],-1)

In [ ]:

# Create annoy indexer 
def create_annoy(preds, metric, n_trees=1000):
    f = 5000
#     start = time.time()
    
    t = AnnoyIndex(f, metric)  # Length of item vector that will be indexed
    for i in range(preds.shape[0]):
        v = preds[i]
        t.add_item(i, v)
    
    t.build(n_trees)
    
#     end = time.time()
#     print(f'Built annoy {metric} model in {end - start} seconds')
    return t

def save_annoy(fileName,t):
    save_dir = os.path.join(annoy_dir,fileName)
    t.save(save_dir)

# Load annoy model
def load_annoy(fileName,metric='euclidean'):
    f = 5000
    load_dir = os.path.join(annoy_dir,fileName)
    t = AnnoyIndex(f, metric)
    t.load(load_dir)
    return t

# Load an image to the current model
def load_img_annoy(t,v,n_trees=1000):
    # Unbuild to add drawing
    start = time.time()
    t.unbuild()
    
    last_index = t.get_n_items()
    t.add_item(last_index, v)
    # 
    t.build(n_trees)
    end = time.time()
    print(f'Reloaded annoy in {end - start} seconds')
    
    return t

In [ ]:
# Build the annoy index from the full dataset (RUN THIS ONCE)   
# Recommend 'euclidean' or 'angular' (closest to cosine)
metric = "euclidean"
annoy_dir = './annoy/'

annoy_indexer = create_annoy(original,metric)

save_annoy(f'annoy-{metric}.ann',annoy_indexer) # 

In [ ]:
# Load the annoy index 
# annoy_indexer = load_annoy(f'annoy-{metric}_(Drawings).ann', metric=metric)

## Plot images

In [ ]:
pairs =[]

plot_names=[]

for new_drawing,drawing_name in tqdm(zip(drawing_prob,draw_labels)):
    
    idx = annoy_indexer.get_nns_by_vector(new_drawing, 8) # will find the nearest neighbors of index 1070
    print(idx)
    
    for i in range(0, 8):
        
        database_name = label_names[idx[i]] 

        database_name = os.path.join(train_dir,database_name)
        
        if i == 0:
            plot_names.append(drawing_name)
        
        plot_names.append(database_name)
        

print(plot_names)      

In [ ]:
fig, axes = plt.subplots(25,9,figsize=(40,100))
[axi.set_axis_off() for axi in axes.ravel()]

for i, (ax,im) in enumerate(zip(axes.flatten(),plot_names)):
    img = PIL.Image.open(im)
    ax.imshow(img,cmap='gray')

plt.subplots_adjust(left=0,right=1,bottom=0,top=1,wspace = 0,hspace = 0)

fig.savefig(os.path.join(test_batch_dir,f'drawings_similarity_{metric}.png'),dpi=150)
